### Installation

In [ ]:
# !pip install openpmd-viewer

### Package imports

- Explain software packages that are used

In [ ]:
from scipy.constants import c, e                          # Physical constants
from openpmd_viewer.addons import LpaDiagnostics as opmd  # OpenPMD Viewer (add-on for plasma acceleration)
import matplotlib.pyplot as plt                           # Plotting

### Analysis

In [ ]:
# Open the diagnostic files and create an OpenPMD TimeSeries object
d = opmd('./lab_diags/hdf5/')

In [ ]:
# Get the laser waist along the propagation
laser_waist = d.iterate( d.get_laser_waist, pol='x')
z_prop = d.t*c

In [ ]:
# Plot the laser waist
plt.figure(figsize=(4,2), dpi=150)
plt.plot(z_prop*1.e6, laser_waist*1.e6)
plt.xlabel('$z_{prop}$ (µm)')
plt.ylabel('$w_{0}$ (µm)')

In [ ]:
# Get the charge along the propagation
charge = d.iterate( d.get_charge )

In [ ]:
# Plot the charge
plt.figure(figsize=(4,2), dpi=150)
plt.plot(z_prop*1.e6, -charge*1.e12, color='steelblue')
plt.xlabel('$z_{prop}$ (µm)')
plt.ylabel('$Q$ (pC)')

In [ ]:
# Get the energy (mean) and energy spread (std) of the bunch along the progation
energy, energy_spread = d.iterate(d.get_energy_spread, center='mean', width='std')

In [ ]:
# Plot the energy
plt.figure(figsize=(4,2), dpi=150)
plt.plot(z_prop*1.e6, energy, color='steelblue')
plt.xlabel('$z_{prop}$ (µm)')
plt.ylabel('$E$ (MeV)')
# Plot the energy spread
plt.twinx()
plt.plot(z_prop*1.e6, energy_spread, color='lightgreen')
plt.xlabel('$z_{prop}$ (µm)')
plt.ylabel('$\delta_E$ (MeV)')

In [ ]:
# Get bunch properties at the plasma exit
select = {'uz': [75,175], 'x':[-3.e-6, 3.e-6], 'y':[-3.e-6, 3.e-6]}
# Get longitduinal phase space
z, uz = d.get_particle(['z', 'uz'], iteration=10, select=select)
# Get the charge
bunch_charge = -d.get_charge(iteration=10, select=select)*1.e12
# Get the median energy and median absolute deviation (mad) energy spread 
median_energy, mad_energy_spread = d.get_energy_spread(iteration=10, center='median', width='mad', select=select)
# Get the emittance
emit_x, emit_y = d.get_emittance(iteration=10, select=select)

In [ ]:
# Plot the bunch phase space and characteristic properties at the end of the propagation
plt.figure(figsize=(4,2), dpi=150)
plt.hist2d(z*1.e6, uz, range=((704,710),(0,175)), bins=100, cmap='YlGnBu_r');
plt.ylim(0,175)
plt.xlabel('$z$ (µm)')
plt.ylabel('$u_z$')
plt.annotate('$Q = %02.02f$ (pC)'%(bunch_charge), xy = (707,80), color='w')
plt.annotate('$E = %02.02f$ (MeV)'%(median_energy), xy = (707,60), color='w')
plt.annotate('$\Delta E_{MAD} = %02.02f$'%(mad_energy_spread/median_energy*100) + ' %', xy = (707,40), color='w')
plt.annotate('$\epsilon_{n,x} = %02.02f$'%(emit_x*1.e6), xy = (707,20), color='w')